In [12]:
#----------------------------
# import libraries
#----------------------------
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
#import sklearn
from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import train_test_split
#from sklearn import preprocessing

In [4]:
def reserve_test_set(df, ratio):
    # shuffle array
    df = sklearn.utils.shuffle(df)
    # calculate number of samples in test set
    N = round( ratio * df.shape[0] )
    # split dataframe
    test_set = df.iloc[:N, :]
    df = df.iloc[N:, :]
    return test_set, df

In [5]:
# path to save figures
path = 'figures/ex1/'
# read data
df = pd.read_csv('dados_voz_genero.csv')
# remove first column (index)
df = df.drop(df.columns[[0]], axis=1)
# dataframe to numpy
features = df.to_numpy()

In [6]:
# split data according to the label
df_male = df[df['label'] == 1]
df_female = df[df['label'] == 0]

In [7]:
ratio = 0.2
# shuffle and reserve set for test
df_test_female, df_female = reserve_test_set(df_female, ratio)
df_test_male, df_male = reserve_test_set(df_male, ratio)

In [8]:
# concatenate and shuffle test set
test_set = [df_test_male, df_test_female]
test_set = pd.concat(test_set)
test_set = sklearn.utils.shuffle(test_set)

In [9]:
# concatenate and shuffle test set
train_set = [df_male, df_female]
train_set = pd.concat(train_set)
train_set = sklearn.utils.shuffle(train_set)

In [10]:
train_set

,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
1335,0.070548,0.192527,0.105911,0.218254,0.112343,1.908920,6.319250,0.936775,0.613574,0.205819,0.164710,0.123410,0.046967,0.265193,1.279576,0.023438,8.109375,8.085938,0.109764,1.0
2630,0.035254,0.203960,0.186931,0.237228,0.050297,2.107986,8.001609,0.859092,0.189103,0.186931,0.208453,0.164184,0.049281,0.279070,1.677329,0.164062,15.234375,15.070312,0.070295,0.0
1678,0.078063,0.209176,0.150212,0.244094,0.093882,5.816900,71.271204,0.915355,0.536783,0.050071,0.186963,0.166523,0.016064,0.275862,0.225204,0.054688,0.781250,0.726562,0.292766,0.0
1241,0.059767,0.176699,0.129398,0.239767,0.110369,0.916961,3.558190,0.941125,0.487553,0.246019,0.181223,0.137259,0.047525,0.275862,1.125000,0.023438,6.117188,6.093750,0.067846,1.0
1133,0.066958,0.172956,0.135788,0.239363,0.103575,1.221574,4.276509,0.945091,0.608689,0.276035,0.179496,0.138419,0.048048,0.279070,1.474276,0.023438,8.578125,8.554688,0.155321,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1951,0.031964,0.198973,0.184434,0.223804,0.039370,2.447280,10.352931,0.864479,0.165662,0.185904,0.202647,0.184609,0.021769,0.250000,1.107759,0.070312,6.140625,6.070312,0.197010,0.0
1686,0.074603,0.182748,0.100992,0.223893,0.122901,5.404833,51.399879,0.924390,0.615591,0.050229,0.168381,0.148348,0.016684,0.275862,0.212612,0.054688,0.593750,0.539062,0.249721,0.0
246,0.092350,0.127601,0.019148,0.203879,0.184731,33.566338,1193.434066,0.829816,0.515830,0.000000,0.125249,0.102683,0.016842,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,1.0
2374,0.037653,0.256402,0.229173,0.265478,0.036305,2.925315,11.767283,0.811305,0.181158,0.256402,0.243266,0.186963,0.051010,0.277457,1.597479,0.023438,9.656250,9.632812,0.116241,0.0


In [ ]:
>>> from sklearn.preprocessing import StandardScaler
>>> data = [[0, 0], [0, 0], [1, 1], [1, 1]]
>>> scaler = StandardScaler()
>>> print(scaler.fit(data))
StandardScaler(copy=True, with_mean=True, with_std=True)
>>> print(scaler.mean_)
[0.5 0.5]
>>> print(scaler.transform(data))
[[-1. -1.]
 [-1. -1.]
 [ 1.  1.]
 [ 1.  1.]]
>>> print(scaler.transform([[2, 2]]))
[[3. 3.]]

In [34]:
# data standardization: x' = (x - u) / s
scaler = StandardScaler()
scaler.fit(train_set)
scaler.transform(train_set)

array([[ 0.80049332,  0.19454469, -0.71038252, ...,  0.87536633,
        -0.54752969,  1.        ],
       [-1.31678061,  0.50671316,  0.95782727, ...,  2.86151126,
        -0.87695904, -1.        ],
       [ 1.25135983,  0.64912627,  0.20177638, ..., -1.21741726,
         0.97994647, -1.        ],
       ...,
       [ 2.10839667, -1.57811001, -2.49685602, ..., -1.42402965,
        -1.46369687,  1.        ],
       [-1.17282697,  1.93851088,  1.8276144 , ...,  1.31525078,
        -0.49346508, -1.        ],
       [-1.02245481,  0.07751616,  0.73119195, ...,  0.39104911,
         1.05718057, -1.        ]])